# Class structure

## Import dependences

In [ ]:
pip install -U scikit-learn scipy matplotlib

In [ ]:
pip install --upgrade tensorflow

In [ ]:
import random
import string
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import datasets, layers, models, optimizers, utils, losses

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

## CnnModel Class

---
Atributes
```
```
---
Methods
```
build(): self
```


In [ ]:
class CnnModel:
    @staticmethod

    def build():
        model = models.Sequential()

        model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))
        model.add(layers.Dropout(0.3))

        model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dropout(0.5))

        model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dropout(0.5))

        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(10, activation='softmax'))

        model.compile(
            optimizer='adam',
            loss=losses.categorical_crossentropy,
            metrics=['accuracy']
        )

        return model

## RandomForestModel Class

---
Atributes
```
```
---
Methods
```
build(): self
```


In [ ]:
class RandomForestModel:
    @staticmethod

    def build():
        return RandomForestClassifier(
            criterion='gini',
            max_depth=10,
            max_features='sqrt',
            min_samples_leaf=10
        )

## KNearestNeighborsModelClass

---
Atributes
```
```
---
Methods
```
build(): self
```


In [ ]:
class KNearestNeighborsModel:
    @staticmethod

    def build():
        return KNeighborsClassifier(
            n_neighbors=3,
            weights='distance',
            algorithm='brute'
        )

## DataBatch Class

---
Atributes

```
_test_images: dict = {}
_test_labels: dict = {}
_train_images: dict = {}
_train_labels: dict = {}
_shards: dict = {}
_num_classes = None
_num_clients = None
```

---
Methods
```
getters(): self
_split_data(id_type): void
```

In [ ]:
class DataBatch:

    # attributes
    _test_images: dict = {}
    _test_labels: dict = {}
    _train_images: dict = {}
    _train_labels: dict = {}
    _shards: dict = {}
    _num_classes = None
    _num_clients = None


    def __init__(self, num_clients, num_classes):
        self._num_clients = num_clients
        self._num_classes = num_classes


    # getters
    def get_test_images(self, id_type):
        return self._test_images[id_type]


    def get_test_labels(self, id_type):
        return self._test_labels[id_type]


    def get_train_images(self, id_type):
        return self._train_images[id_type]


    def get_train_labels(self, id_type):
        return self._train_labels[id_type]


    def get_shards(self, id_type):
        return self._shards[id_type]


    def get_full_shards(self):
        return self._shards


    # methods
    def _split_data_iid(self, id_type):
        data = list(zip(
            self._train_images[id_type],
            self._train_labels[id_type]
        ))
        random.shuffle(data)
        train, label = zip(*data)

        len_shards = len(self._train_images[id_type]) // self._num_clients
        range_start_shard = 0
        range_final_shard = len_shards

        shards = {}
        i = 0
        for i in range(self._num_clients):
            shards[i] = {
                'train_images': [],
                'train_labels': []
            }

            shards[i]['train_images'] = train[range_start_shard:range_final_shard]
            shards[i]['train_labels'] = label[range_start_shard:range_final_shard]

            range_start_shard = range_final_shard
            range_final_shard = range_final_shard + len_shards

        self._shards[id_type] = shards
    
    
    def _split_data_non_iid(self, id_type):
        data = list(zip(
            self._train_images[id_type],
            self._train_labels[id_type]
        ))
        
        if id_type == 'cnn':
            data.sort(key=lambda e: np.argmax(e[1]))
        else:
            data.sort(key=lambda e: e[1])
        
        x_order_train, y_order_train = zip(*data)
        
        shards = {}
        i = 0
        for i in range(self._num_clients):
            j = 0
            shards[i] = {
                'train_images': [],
                'train_labels': [],
                'data_count': {j: 0 for j in range(self._num_classes)}
            }
        
        label_proportion = (.38, .12, .12, .09, .09, .05, .05, .04, .03, .03)

        assert self._num_clients == len(label_proportion)
        assert sum(label_proportion) == 1

        len_label_sample = len(x_order_train) // self._num_clients

        c = start = end = 0
        for c in range(self._num_clients):
            key_shard = c

            i = p = 0
            for i, p in enumerate(label_proportion):
                start = end
                end = start + int(len_label_sample * p)

                if key_shard == self._num_clients:
                    key_shard = 0

                shards[key_shard]['train_images'] += x_order_train[start:end]
                shards[key_shard]['train_labels'] += y_order_train[start:end]

                key_shard += 1

        self._shards[id_type] = shards
        
        
    def _show_distribution(self, id_type):
        i = 0
        for i in range(self._num_clients):

            j = y = 0
            for j, y in enumerate(self._shards[id_type][i]['train_labels']):
                if id_type == 'cnn':
                    self._shards[id_type][i]['data_count'][(np.argmax(y))] += 1
                else:
                    self._shards[id_type][i]['data_count'][y] += 1

            print(
                len(self._shards[id_type][i]['train_images']),
                len(self._shards[id_type][i]['train_labels']),
                self._shards[id_type][i]['data_count']
            )

## CnnData(DataBatch) Class

---
Atributes

```
__TYPE = 'cnn'
```

---
Methods
```
__prepare_data_to_cnn: void
```

In [ ]:
class CnnData(DataBatch):

    # attributes
    __TYPE = 'cnn'


    def __init__(self, dataset, num_clients, num_classes):
        super().__init__(num_clients, num_classes)
        (self._train_images[self.__TYPE], self._train_labels[self.__TYPE]), (self._test_images[self.__TYPE], self._test_labels[self.__TYPE]) = dataset
        self.__prepare_data_to_cnn()
        self._split_data_non_iid(self.__TYPE)
        self._show_distribution(self.__TYPE)


    # methods
    def __prepare_data_to_cnn(self):
        self._train_labels[self.__TYPE] = utils.to_categorical(
            self._train_labels[self.__TYPE],
            self._num_classes
        )

        self._test_labels[self.__TYPE] = utils.to_categorical(
            self._test_labels[self.__TYPE],
            self._num_classes
        )

        self._train_images[self.__TYPE] = self._train_images[self.__TYPE].astype('float32')
        self._test_images[self.__TYPE] = self._test_images[self.__TYPE].astype('float32')

        self._train_images[self.__TYPE] /= 255.
        self._test_images[self.__TYPE] /= 255.

## RandomForestData(DataBatch) Class

---
Atributes

```
__TYPE = 'rf'
```

---
Methods
```
__prepare_data_to_rf: void
```

In [ ]:
class RandomForestData(DataBatch):

    # attributes
    __TYPE = 'rf'


    def __init__(self, dataset, num_clients, num_classes):
        super().__init__(num_clients, num_classes)
        (self._train_images[self.__TYPE], self._train_labels[self.__TYPE]), (self._test_images[self.__TYPE], self._test_labels[self.__TYPE]) = dataset
        self.__prepare_data_to_rf()
        self._split_data_non_iid(self.__TYPE)
        self._show_distribution(self.__TYPE)


    # methods
    def __prepare_data_to_rf(self):
        self._train_labels[self.__TYPE] = self._train_labels[self.__TYPE].flatten()
        self._test_labels[self.__TYPE] = self._test_labels[self.__TYPE].flatten()

        self._train_images[self.__TYPE] = self._train_images[self.__TYPE].reshape(50000, 3*32*32)
        self._test_images[self.__TYPE] = self._test_images[self.__TYPE].reshape(10000, 3*32*32)

        self._train_images[self.__TYPE] = self._train_images[self.__TYPE].astype('float32')
        self._test_images[self.__TYPE] = self._test_images[self.__TYPE].astype('float32')

        self._train_images[self.__TYPE] /= 255.
        self._test_images[self.__TYPE] /= 255.

## KNearestNeighborsData(DataBatch) Class

---
Atributes

```
__TYPE = 'knn'
```

---
Methods
```
__prepare_data_to_knn: void
```

In [ ]:
class KNearestNeighborsData(DataBatch):

    # attributes
    __TYPE = 'knn'


    def __init__(self, dataset, num_clients, num_classes):
        super().__init__(num_clients, num_classes)
        (self._train_images[self.__TYPE], self._train_labels[self.__TYPE]), (self._test_images[self.__TYPE], self._test_labels[self.__TYPE]) = dataset
        self.__prepare_data_to_knn()
        self._split_data_non_iid(self.__TYPE)
        self._show_distribution(self.__TYPE)


    # methods
    def __prepare_data_to_knn(self):
        self._train_labels[self.__TYPE] = self._train_labels[self.__TYPE].flatten()
        self._test_labels[self.__TYPE] = self._test_labels[self.__TYPE].flatten()

        self._train_images[self.__TYPE] = self._train_images[self.__TYPE]/255.0
        self._test_images[self.__TYPE] = self._test_images[self.__TYPE]/255.0

        n_samples, nx, ny, nrgb = self._train_images[self.__TYPE].shape
        self._train_images[self.__TYPE] = self._train_images[self.__TYPE].reshape((n_samples, nx*ny*nrgb)) # (50000, 32*32*3)

        n_samples, nx, ny, nrgb = self._test_images[self.__TYPE].shape
        self._test_images[self.__TYPE] = self._test_images[self.__TYPE].reshape((n_samples, nx*ny*nrgb)) # (10000, 32*32*3)

## ClientHost Class

---
Atributes
```
name: str
data
model
prediction
history
```
---
Methods
```
getters(): self
setters(self): void
fit_model(id_type, test_images, test_labels): void
plot_accuracy(id_type): void
plot_loss(id_type): void
print_details_train(): void
make_predict(id_type, test_sample): void
```


In [ ]:
class ClientHost:
    
    # attributes
    __name: str = ''
    __data_batches: dict = {}
    __models: dict = {}
    __histories: dict = {}
    __predictions: dict = {}


    def __init__(self):
        self.__name = ''.join(
            random.choice(
                string.ascii_uppercase + string.digits
            ) for _ in range(6)
        )


    # getters
    @property
    def name(self):
        return self.__name

    @property
    def data_batches(self):
        return self.__data_batches

    @property
    def models(self):
        return self.__models

    @property
    def histories(self):
        return self.__histories
    
    @property
    def predictions(self):
        return self.__predictions


    # setters
    @data_batches.setter
    def data_batches(self, data_batches):
        self.__data_batches = data_batches

    @models.setter
    def models(self, models):
        self.__models = models
    
    @histories.setter
    def histories(self, histories):
        self.__histories = histories

    @predictions.setter
    def predictions(self, predictions):
        self.__predictions = predictions

        
    # class methods
    @staticmethod
    def fit_model(id_type, model, train_images, train_labels, test_images=None, test_labels=None, epochs=None):
        data_batch = {}
        
        if id_type == 'cnn':
            history = model.fit(
                x=tf.stack(train_images),
                y=tf.stack(train_labels),
                epochs=epochs,
                validation_data=(
                    test_images,
                    test_labels
                )
            )
            print(id_type,
                  np.argmax(train_labels[0]),
                  np.argmax(train_labels[2500]),
                  np.argmax(train_labels[4999])
            )
        else:
            history = model.fit(
                X=train_images,
                y=train_labels
            )
            print(id_type, train_labels[0], train_labels[2500], train_labels[4999])
            
        data_batch = {
            'train_images': train_images,
            'train_labels': train_labels
        }
        
        return history, model, data_batch
    

    @staticmethod
    def make_predict(id_type, model, test_sample):
        predictions = []

        for sample in test_sample:
            image = np.expand_dims(sample['image'], 0)
            prediction = model.predict(image)
            
            if id_type == 'cnn':
                predict = np.argmax(prediction[0])
                full_predict = prediction[0]
            else:
                predict = prediction[0]
                full_predict = model.predict_proba(image)

            predictions.append({
                'label': sample['label'],
                'predict': predict,
                'full_predict': full_predict
            })
        
        return predictions
    
    
    # def plot_accuracy(self, id_type):
    #   plt.figure(figsize=[6,4])
    #   plt.plot(self.__histories[id_type].history['accuracy'], 'black', linewidth=2.0)
    #   plt.plot(self.__histories[id_type].history['val_accuracy'], 'blue', linewidth=2.0)
    #   plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=14)
    #   plt.xlabel('Epochs', fontsize=10)
    #   plt.ylabel('Accuracy', fontsize=10)
    #   plt.title('Accuracy Curves', fontsize=12)
    #   plt.show()


    # def plot_loss(self, id_type):
    #   plt.figure(figsize=[6,4])
    #   plt.plot(self.__histories[id_type].history['loss'], 'black', linewidth=2.0)
    #   plt.plot(self.__histories[id_type].history['val_loss'], 'green', linewidth=2.0)
    #   plt.legend(['Training Loss', 'Validation Loss'], fontsize=14)
    #   plt.xlabel('Epochs', fontsize=10)
    #   plt.ylabel('Loss', fontsize=10)
    #   plt.title('Loss Curves', fontsize=12)
    #   plt.show()


    # def print_details_train(self):
    #   test_loss, test_acc = self.__models.evaluate(
    #     self._test_images,
    #     self._test_labels,
    #     verbose=2
    #   )

    #   print(
    #     'Client: {} | accuracy: {:.2%} | loss: {} \n\n\n'.format(
    #       self._name,
    #       test_acc,
    #       test_loss
    #     )
    #   )

## ConsensusMechanisms Class

---
Atributes
```
```
---
Methods
```
## public ##
plotHitRate(): void
makeMajority(): void
makeWeightedMajority(): void
makeBordaCount(): void
```


In [ ]:
class ConsensusMechanisms:
    
    # class methods
    def __is_unanimous(self, voting_list):
        count = 0
        for key, value in voting_list.items():
            if value > 0:
                count = count + 1

        return count == 1
    
    
    def plot_hit_rate(self, voting):
        hits = 0

        for vote in voting:
            if vote['correct']:
                hits += 1

        if hits > 0:
            return 'Hit rate {:.4%}'.format((hits / len(voting)))
        else:
            return 'Hit rate 0%'


    def make_majority(self, type_sample, test_sample, client_list):
        majority_voting = list()

        for label_id, sample in enumerate(test_sample):
            
            if type(sample['label']) is np.ndarray:
                label_value = np.argmax(sample['label'])
            else:
                label_value = sample['label']

            i = 0
            voting = {i: 0 for i in range(10)}

            correct = False
            highest_value = 0

            for client in client_list:
                client_prediction = client.predictions[type_sample][label_id]
                voting[client_prediction['predict']] += 1

            highest_value = max(voting, key=voting.get)
            
            if label_value == highest_value:
                correct = True

            unanimous = self.__is_unanimous(voting)

            majority_voting.append(
                {
                    'target': label_value,
                    'correct': correct,
                    'unanimous': unanimous,
                    'voting': voting
                }
            )

        return majority_voting

  
    def make_weighted_majority(self, type_sample, test_sample, client_list):
        weighted_majority_voting = list()
        len_clients = len(client_list)

        for label_id, sample in enumerate(test_sample):
            
            if type(sample['label']) is np.ndarray:
                label_value = np.argmax(sample['label'])
            else:
                label_value = sample['label']

            i = 0
            voting = {i: 0 for i in range(10)}
            correct = False

            for client in client_list:
                client_prediction = client.predictions[type_sample][label_id]
                npArgmax = client_prediction['predict']

                if len(client_prediction['full_predict']) == 1:
                    certainty_level = client_prediction['full_predict'][0][npArgmax]
                else:
                    certainty_level = client_prediction['full_predict'][npArgmax]

                certainty_coefficient = certainty_level + 1
                voting[npArgmax] += (certainty_coefficient * certainty_level)

            if label_value == max(voting, key=voting.get):
                correct = True
            
            unanimous = self.__is_unanimous(voting)

            weighted_majority_voting.append(
                {
                    'target': label_value,
                    'correct': correct,
                    'unanimous': unanimous,
                    'voting': voting
                }
            )

        return weighted_majority_voting

  
    def make_borda_count(self, type_sample, test_sample, client_list):
        borda_count_voting = list()
        len_clients = len(client_list)
        clients_ballots = list()

        for client in client_list:
            
            client_ballot = {
                'client': client.name,
                'ballots': list()
            }

            for predictions in client.predictions[type_sample]:
                ballot = list()
                
                if len(predictions['full_predict']) == 1:
                    full_predict = predictions['full_predict'][0]
                else:
                    full_predict = predictions['full_predict']
                    
                for option, confidence in enumerate(full_predict):
                    ballot.append({
                        'option': option,
                        'confidence': confidence
                    })

                ballot.sort(reverse=True, key=lambda e: e['confidence'])
                max_weight = len(ballot)

                for coefficient, vote in enumerate(ballot):
                    vote['weight'] = max_weight - coefficient

                client_ballot['ballots'].append(ballot)

            clients_ballots.append(client_ballot)


        for label_id, sample in enumerate(test_sample):
            
            if type(sample['label']) is np.ndarray:
                label_value = np.argmax(sample['label'])
            else:
                label_value = sample['label']

            i = 0
            voting = {i: 0 for i in range(10)}
            correct = False

            for client in clients_ballots:
                for item_ballot in client['ballots'][label_id]:
                    voting[item_ballot['option']] += item_ballot['weight']

            if label_value == max(voting, key=voting.get):
                correct = True
            
            unanimous = self.__is_unanimous(voting)

            borda_count_voting.append({
                'target': label_value,
                'correct': correct,
                'unanimous': unanimous,
                'voting': voting
            })

        return borda_count_voting

# Application excution

## Prepapre dataset

In [ ]:
CLIENT_RANGE = 10
CLASSES_RANGE = 10
loaded_data = datasets.cifar10.load_data()


print('CNN')
dataset = CnnData(loaded_data, CLIENT_RANGE, CLASSES_RANGE)
print('RF')
dataset = RandomForestData(loaded_data, CLIENT_RANGE, CLASSES_RANGE)
print('KNN')
dataset = KNearestNeighborsData(loaded_data, CLIENT_RANGE, CLASSES_RANGE)


print('CNN dataset test len:', len(dataset.get_test_images('cnn')))        # 10.000
print('RF dataset test len:', len(dataset.get_test_images('rf')))          # 10.000
print('KNN dataset test len:', len(dataset.get_test_images('knn')))        # 10.000

print('CNN dataset train len:', len(dataset.get_train_images('cnn')))      # 50.000
print('RF dataset train len:', len(dataset.get_train_images('rf')))        # 50.000
print('KNN dataset train len:', len(dataset.get_train_images('knn')))      # 50.000

print('CNN total clients:', len(dataset.get_shards('cnn')))
print('RF total clients:', len(dataset.get_shards('rf')))
print('KNN total clients:', len(dataset.get_shards('knn')))

## Make sample CNN data

In [ ]:
CNN_IMAGES = dataset.get_test_images('cnn')
CNN_LABELS = dataset.get_test_labels('cnn')

cnn_test_sample = list()

# SAMPLE_LENGTH = 100   # 100
# SAMPLE_LENGTH = int(len(CNN_IMAGES) * 0.1)   # 10%
SAMPLE_LENGTH = len(CNN_IMAGES)   # 100%

for i in range(SAMPLE_LENGTH):
    cnn_test_sample.append({
        'image': CNN_IMAGES[i],
        'label': CNN_LABELS[i]
    })

print(SAMPLE_LENGTH, len(cnn_test_sample))

## Make sample RF data

In [ ]:
RF_IMAGES = dataset.get_test_images('rf')
RF_LABELS = dataset.get_test_labels('rf')

rf_test_sample = list()

# SAMPLE_LENGTH = 100   # 100
# SAMPLE_LENGTH = int(len(RF_IMAGES) * 0.1)   # 10%
SAMPLE_LENGTH = len(RF_IMAGES)   # 100%

for i in range(SAMPLE_LENGTH):
    rf_test_sample.append({
        'image': RF_IMAGES[i],
        'label': RF_LABELS[i]
    })

print(SAMPLE_LENGTH, len(rf_test_sample))

## Make sample KNN data

In [ ]:
KNN_IMAGES = dataset.get_test_images('knn')
KNN_LABELS = dataset.get_test_labels('knn')

knn_test_sample = list()

# SAMPLE_LENGTH = 100   # 100
# SAMPLE_LENGTH = int(len(KNN_IMAGES) * 0.1)   # 10%
SAMPLE_LENGTH = len(KNN_IMAGES)   # 100%

for i in range(SAMPLE_LENGTH):
    knn_test_sample.append({
        'image': KNN_IMAGES[i],
        'label': KNN_LABELS[i]
    })

print(SAMPLE_LENGTH, len(knn_test_sample))

## Prepare clients (federate nodes)

In [ ]:
client_list = []

i = 0
for i in range(CLIENT_RANGE):
    client_list.append(ClientHost())


i = 0
for i, client in enumerate(client_list):
    
    histories = {}
    client_models = {}
    data_batches = {}
    predictions = {}
    
    print(i, '-', client.name)

    print('To CNN')
    
    history, model, data_batch = client.fit_model(
        id_type='cnn',
        model=CnnModel().build(),
        train_images=dataset.get_shards('cnn')[i]['train_images'],
        train_labels=dataset.get_shards('cnn')[i]['train_labels'],
        test_images=dataset.get_test_images('cnn'),
        test_labels=dataset.get_test_labels('cnn'),
        epochs=100
    )
    
    predict = client.make_predict(
        id_type='cnn',
        model=model,
        test_sample=cnn_test_sample
    )
    
    histories['cnn'] = history
    client_models['cnn'] = model
    data_batches['cnn'] = data_batch
    predictions['cnn'] = predict
    
    # client.plot_accuracy('cnn')
    # client.plot_loss('cnn')
    print('=====\n')


    print('To Random Forest')
    
    history, model, data_batch = client.fit_model(
        id_type='rf',
        model=RandomForestModel().build(),
        train_images=dataset.get_shards('rf')[i]['train_images'],
        train_labels=dataset.get_shards('rf')[i]['train_labels']
    )
    
    predict = client.make_predict(
        id_type='rf',
        model=model,
        test_sample=rf_test_sample
    )
    
    histories['rf'] = history
    client_models['rf'] = model
    data_batches['rf'] = data_batch
    predictions['rf'] = predict
    
    # client.plot_accuracy('rf')
    # client.plot_loss('rf')
    print('=====\n')
    
    
    print('To K-Nearest Neighbors')
    
    history, model, data_batch = client.fit_model(
        id_type='knn',
        model=KNearestNeighborsModel().build(),
        train_images=dataset.get_shards('knn')[i]['train_images'],
        train_labels=dataset.get_shards('knn')[i]['train_labels']
    )
    
    predict = client.make_predict(
        id_type='knn',
        model=model,
        test_sample=knn_test_sample
    )
    
    histories['knn'] = history
    client_models['knn'] = model
    data_batches['knn'] = data_batch
    predictions['knn'] = predict
    
    # client.plot_accuracy('knn')
    # client.plot_loss('knn')
    
    print('=====\n')
    

    client.histories = histories
    client.models = client_models
    client.data_batches = data_batches
    client.predictions = predictions

## Execute

In [ ]:
consensus_mechanisms = ConsensusMechanisms()

types_sample = [
    'cnn',
    'rf',
    'knn'
]

majority_voting = {}
weighted_majority_voting = {}
borda_count_voting = {}

for type_sample in types_sample:
    print('To', type_sample)

    # --
    majority_voting[type_sample] = consensus_mechanisms.make_majority(
        type_sample,
        globals().get(type_sample + '%s' % '_test_sample'),
        client_list
    )
    print('Majority', consensus_mechanisms.plot_hit_rate(majority_voting[type_sample]))

    # --
    weighted_majority_voting[type_sample] = consensus_mechanisms.make_weighted_majority(
        type_sample,
        globals().get(type_sample + '%s' % '_test_sample'),
        client_list
    )
    print('Weighted Majority', consensus_mechanisms.plot_hit_rate(weighted_majority_voting[type_sample]))

    # --
    borda_count_voting[type_sample] = consensus_mechanisms.make_borda_count(
        type_sample,
        globals().get(type_sample + '%s' % '_test_sample'),
        client_list
    )
    print('Borda Count', consensus_mechanisms.plot_hit_rate(borda_count_voting[type_sample]))

    print('\n')



for i in random.sample(range(100), k=5):
    print('Index sample', i)
    print('\n')
    
    print('To cnn')
    print('Majority', majority_voting['cnn'][i])
    print('Weighted Majority', weighted_majority_voting['cnn'][i])
    print('Borda Count', borda_count_voting['cnn'][i])
    print('\n')

    print('To rf')
    print('Majority', majority_voting['rf'][i])
    print('Weighted Majority', weighted_majority_voting['rf'][i])
    print('Borda Count', borda_count_voting['rf'][i])
    print('\n')
    
    print('To knn')
    print('Majority', majority_voting['knn'][i])
    print('Weighted Majority', weighted_majority_voting['knn'][i])
    print('Borda Count', borda_count_voting['knn'][i])
    print('\n')

In [ ]:
types_sample = [
    'cnn',
    'rf',
    'knn'
]

for client in client_list:
    print('Client: ', client.name)
    
    for type_sample in types_sample:
        print(client.models[type_sample])
        
        if type_sample != 'cnn':
            print(
                'Score model: {:.2%}'.format(
                    client.models[type_sample].score(
                        dataset.get_test_images(type_sample),
                        dataset.get_test_labels(type_sample)
                    )
                )
            )
        else:
            print(
                'Score model: {:.2%}'.format(
                    client.models[type_sample].evaluate(
                        dataset.get_test_images(type_sample),
                        dataset.get_test_labels(type_sample),
                        verbose=0
                    )[1]
                )
            )
            
        
    print('\n') 

In [ ]:
LABELS_TEST = KNN_LABELS


def normalize_decisions_of_models(ballot, models=['cnn', 'rf', 'knn']):
    result = {}
    
    for model in models:        
        for label, voting in enumerate(ballot[model]):
            result.setdefault(label, []).append({
                model: max(voting['voting'], key=voting['voting'].get),
                'ballot': voting['voting']
            })
        
    return result


def get_models_decisions(decisions_normalized, models=['cnn', 'rf', 'knn']):
    
    chosen_labels = []
    
    for index, results in decisions_normalized.items():
        chosen_ones = {}

        for result in results:
            value = max(result['ballot'], key=result['ballot'].get)
            chosen_ones.setdefault(value, 0)
            chosen_ones[value] += 1

        if len(chosen_ones) == len(models):
            undecideds = {}

            for result in results:
                value = max(result['ballot'], key=result['ballot'].get)
                qtd_votes = result['ballot'][value]
                undecideds[value] = qtd_votes

            chosen_ones = undecideds
        
        chosen_labels.append(max(chosen_ones, key=chosen_ones.get))
    
    return chosen_labels
        

    
    
hits = 0
print('\033[1mMAJORITY VOTING\033[0m\n')
for i, chosen in enumerate(get_models_decisions(normalize_decisions_of_models(ballot=majority_voting))):
    print(
        'Label: ', i,
        'Expected: ', LABELS_TEST[i],
        'Result: ', chosen,
        'Status: ', ('Correct' if chosen == LABELS_TEST[i] else 'Incorrect')
    )
    
    hits += 1 if chosen == LABELS_TEST[i] else 0
    
print('Hit rate {:.4%}'.format((hits / len(LABELS_TEST))))


hits = 0
print('\n\n\033[1mWEIGHTED MAJORITY VOTING\033[0m\n')
for i, chosen in enumerate(get_models_decisions(normalize_decisions_of_models(ballot=weighted_majority_voting))):
    print(
        'Label: ', i,
        'Expected: ', LABELS_TEST[i],
        'Result: ', chosen,
        'Status: ', ('Correct' if chosen == LABELS_TEST[i] else 'Incorrect')
    )
    
    hits += 1 if chosen == LABELS_TEST[i] else 0
    
print('Hit rate {:.4%}'.format((hits / len(LABELS_TEST))))


hits = 0
print('\n\n\033[1mBORDA COUNT VOTING\033[0m\n')
for i, chosen in enumerate(get_models_decisions(normalize_decisions_of_models(ballot=borda_count_voting))):
    print(
        'Label: ', i,
        'Expected: ', LABELS_TEST[i],
        'Result: ', chosen,
        'Status: ', ('Correct' if chosen == LABELS_TEST[i] else 'Incorrect')
    )
    
    hits += 1 if chosen == LABELS_TEST[i] else 0
    
print('Hit rate {:.4%}'.format((hits / len(LABELS_TEST))))